In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [114]:
def load_dataset():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [115]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [116]:
print(train_set_x_orig.shape)
print(test_set_x_orig.shape)

(209, 64, 64, 3)
(50, 64, 64, 3)


In [117]:
train_set_x_flatten = train_set_x_orig.reshape(64*64*3,209)
test_set_x_flatten = test_set_x_orig.reshape(64*64*3,50)

In [118]:
print(train_set_x_flatten.shape)
print(test_set_x_flatten.shape)

(12288, 209)
(12288, 50)


for a normal dataset we apply feature scailing like normalization or standardization but this is image dataset here we will
divide each and every value of matrics by 255 because the values of 3 classes i.e, RGB are in range 0 to 255

In [119]:
train_set_x = train_set_x_flatten/255
test_set_x = test_set_x_flatten/255

In [120]:
def sigmoid(z):
    a = 1/(1+np.exp(-z))
    return a

In [121]:
def initialize(dim):
    w = np.zeros((dim,1))
    b = 0
    assert(w.shape == (dim,1))
    assert(isinstance(b,float) or isinstance(b,int))
    return w,b

In [122]:
def propogate(x,y,w,b):
    m = x.shape[0]
    A = sigmoid(np.dot(w.T,x)+b)
    cost = np.sum((-y*np.log(A))+(-(1-y)*np.log(1-A)))/m
    dz = A - y
    dw = np.dot(x,dz.T)
    db = np.sum(dz)
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {'dw':dw,'db':db}
    return grads, cost

In [123]:
def optimize(x,y,alpha, iterations,w,b,print_cost=False):
    costs=[]
    for i in range(iterations):
        grads, cost = propogate(x,y,w,b)
        dw = grads['dw']
        db = grads['db']

        w = w - (alpha*dw)
        b = b - (alpha*db)
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    

        params = {'w':w,'b':b}
        grads = {'dw':dw, 'db':db}
    return params,grads,costs

In [124]:
def predict(x,y,w,b):
    y_pred = np.zeros((1,x.shape[0]))
    A = sigmoid(np.dot(w.T,x)+b)
    
    y_pred = (A>=0.5)*1
    assert(y_pred.shape == (1, x.shape[1]))
    return y_pred

In [127]:
def model(x,y,alpha,iterations):
    w,b = initialize(x.shape[0])
    params,grads,cost = optimize(x,y,alpha, iterations,w,b)
    w = params['w']
    b = params['b']
    y_pred = predict(x,y,w,b)
    
    print(100 - np.mean(np.abs(y_pred - y)) * 100)

In [129]:
y_pred = model(train_set_x,train_set_y,0.005,2000)

C:\Users\SUHAS\AppData\Local\Temp\ipykernel_1312\2054185562.py:4: RuntimeWarning: divide by zero encountered in log
  cost = np.sum((-y*np.log(A))+(-(1-y)*np.log(1-A)))/m
C:\Users\SUHAS\AppData\Local\Temp\ipykernel_1312\2054185562.py:4: RuntimeWarning: invalid value encountered in multiply
  cost = np.sum((-y*np.log(A))+(-(1-y)*np.log(1-A)))/m


68.89952153110048
